In [3]:
import gym
from collections import namedtuple
import numpy as np
from tensorboardX import SummaryWriter

import torch
import torch.nn as nn
import torch.optim as optim

## Cross-entropy on CartPole
model's core is a one-hidden-layer neural network, with ReLU and 128 hidden neurons.  
the count of episodes we play on every iteration is 16  
We'll leave the top 30% of episodes sorted by reward == elite episodes

In [4]:
HIDDEN_SIZE = 128
BATCH_SIZE = 16
PERCENTILE = 70

In [6]:
class Net(nn.Module):
    def __init__(self, obs_size, hidden_size, n_actions):
        super(Net, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(obs_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, n_actions)
        )

    def forward(self, x):
        return self.net(x)

환경으로부터 하나의 관측값을 입력벡터 형태로 가져오고, 우리가 행하는 매 행동이 스칼라 형태로 출력값이 나온다.  
소프트맥스를 가장 마지막 층 다음에 추가시키는 것이 직관적인 방법이라고 할 수 있지만, 훈련과정에서의 수치적인 안정성을 위해서 소프트맥스를 추가하지 않았다. 대신에 Cross-entropy loss를 계산한다.

In [7]:
Episode = namedtuple('Episode', field_names=['reward', 'steps'])
EpisodeStep = namedtuple('EpisodeStep', field_names=['observation', 'action'])

In [8]:
def iterate_batches(env, net, batch_size):
    batch = []
    episode_reward = 0.0
    episode_steps = []
    obs = env.reset()
    sm = nn.Softmax(dim=1)
    while True:
        ## 매 반복마다, 우리는 현 관찰값을 PyTorch Tensor로 변환하고
        ## 텐서를 네트워크에 전달한다. 그러면 Action 확률들을 구할 수 있다.
        obs_v = torch.FloatTensor([obs])
        act_probs_v = sm(net(obs_v))  # 네트워크와 소프트맥스를 통과하면 경사를 따라 만들어진 텐서를 낸다
        act_probs = act_probs_v.data.numpy()[0]  # 출력 텐서를 tensor.data로 unpack하고 Numpy array로 변환한다.
        
        ## Action들의 확률분포를 구할 수 있고, 이것을 가지고 random.choice로 샘플링해서 실제 action을 뽑는다. 
        action = np.random.choice(len(act_probs), p=act_probs)
        next_obs, reward, is_done, _ = env.step(action) # action을 환경에 전달해서 다음 관측값과, 보상을 얻는다.
        
        # 보상은 계속 누적되며 episode_step 리스트에 (관측값, action)이 계속 추가된다.
        episode_reward += reward
        episode_steps.append(EpisodeStep(observation=obs, action=action)) # 튜플로 저장되는 것은 action의 결과로 발생되는 관측값이 아니라, action을 선택할 때 사용된 관측값이 저장된다.
        
        
        if is_done:
            batch.append(Episode(reward=episode_reward, steps=episode_steps))
            episode_reward = 0.0  # 보상을 초기화한다.
            episode_steps = []
            next_obs = env.reset()  # 환경을 리셋한다.
            if len(batch) == batch_size:
                yield batch   # 반복문이 실행되는 중간중간에 batch 값을 전달해주기 위한 구문
                batch = []    # batch를 초기화 시킨다.
        obs = next_obs 

In [9]:
def filter_batch(batch, percentile):
    ## 교차엔트로피 방법에서 가장 핵심적인 함수
    ## 에피소드에서 주어진 배치와 퍼센티지 값을 가지고 경계 보상을 구한다.
    ## 모니터링 용도로 평균 보상을 계산한다.
    rewards = list(map(lambda s: s.reward, batch))
    reward_bound = np.percentile(rewards, percentile)
    reward_mean = float(np.mean(rewards))
    
    # 배치안에 있는 모든 에피소드들이 경계값보다 높은 보상을 갖고 있는지 확인한다.
    # 그렇다면 관측값 List와 Action List에 각각 추가한다(각 Space를 확장)

    train_obs = []
    train_act = []
    for example in batch:
        if example.reward < reward_bound:
            continue
        train_obs.extend(map(lambda step: step.observation, example.steps))
        train_act.extend(map(lambda step: step.action, example.steps))
        
    ## 엘리트 에피소드로부터 나온 관측값들과 action들을 텐서로 전환해준다.

    train_obs_v = torch.FloatTensor(train_obs)
    train_act_v = torch.LongTensor(train_act)
    
    # 네개의 원소를 가진 튜플을 반환한다.
    return train_obs_v, train_act_v, reward_bound, reward_mean

In [10]:
env = gym.make("CartPole-v0")  ## 환경 객체를 만든다
# env = gym.wrappers.Monitor(env, directory="mon", force=True)
obs_size = env.observation_space.shape[0]
n_actions = env.action_space.n
net = Net(obs_size, HIDDEN_SIZE, n_actions) ## 신경망 객체를 만든다.
objective = nn.CrossEntropyLoss()  ## 목적함수 객체를 만든다.
optimizer = optim.Adam(params=net.parameters(), lr=0.01) # 옵티마이저 객체를 만든다
writer = SummaryWriter(comment="-cartpole")  # 텐서보드용 SummaryWriter를 만든다.

In [11]:
for iter_no, batch in enumerate(iterate_batches(env, net, BATCH_SIZE)):
    obs_v, acts_v, reward_b, reward_m = filter_batch(batch, PERCENTILE)  # 배치 필터링
    optimizer.zero_grad()
    action_scores_v = net(obs_v) # 관측값을 신경망에 통과시킨다 --> action score가 나온다
    loss_v = objective(action_scores_v, acts_v) # action score와 실제 agent가 취한 action을 목적함수에 넣으면 교차엔트로피 손실값이 나온다.
    loss_v.backward() # gradient를 구한다.
    optimizer.step() # 옵티마이저로 하여금 네트워크를 수정하게 한다.
    print("%d: loss=%.3f, reward_mean=%.1f, reward_bound=%.1f" % (iter_no, loss_v.item(), reward_m, reward_b))
    writer.add_scalar("loss", loss_v.item(), iter_no)
    writer.add_scalar("reward_bound", reward_b, iter_no)
    writer.add_scalar("reward_mean", reward_m, iter_no)
    if reward_m > 199:
        print("Solved!")
        break

0: loss=0.678, reward_mean=23.9, reward_bound=30.5
1: loss=0.683, reward_mean=22.1, reward_bound=28.5
2: loss=0.692, reward_mean=22.1, reward_bound=24.5
3: loss=0.675, reward_mean=29.9, reward_bound=32.0
4: loss=0.658, reward_mean=29.4, reward_bound=34.0
5: loss=0.652, reward_mean=31.2, reward_bound=37.0
6: loss=0.638, reward_mean=41.4, reward_bound=49.5
7: loss=0.650, reward_mean=35.4, reward_bound=35.5
8: loss=0.637, reward_mean=42.4, reward_bound=45.0
9: loss=0.624, reward_mean=34.8, reward_bound=40.0
10: loss=0.608, reward_mean=38.6, reward_bound=48.5
11: loss=0.626, reward_mean=49.9, reward_bound=59.5
12: loss=0.606, reward_mean=43.4, reward_bound=54.5
13: loss=0.618, reward_mean=61.2, reward_bound=67.0
14: loss=0.610, reward_mean=73.7, reward_bound=80.5
15: loss=0.598, reward_mean=70.8, reward_bound=86.5
16: loss=0.598, reward_mean=72.2, reward_bound=99.5
17: loss=0.589, reward_mean=91.2, reward_bound=113.5
18: loss=0.589, reward_mean=84.5, reward_bound=96.5
19: loss=0.604, rewar

In [12]:
writer.close()

In [1]:
!xvfb-run -s "-screen 0 640x480x24" ./01_cartpole.py

0: loss=0.675, reward_mean=25.9, reward_bound=30.0
1: loss=0.674, reward_mean=27.4, reward_bound=39.0
2: loss=0.644, reward_mean=40.6, reward_bound=53.0
3: loss=0.639, reward_mean=46.3, reward_bound=53.0
4: loss=0.626, reward_mean=49.6, reward_bound=61.5
5: loss=0.625, reward_mean=51.9, reward_bound=55.5
6: loss=0.603, reward_mean=57.4, reward_bound=68.0
7: loss=0.600, reward_mean=59.2, reward_bound=65.5
8: loss=0.594, reward_mean=67.4, reward_bound=77.0
9: loss=0.575, reward_mean=69.4, reward_bound=73.5
10: loss=0.570, reward_mean=81.4, reward_bound=106.0
11: loss=0.576, reward_mean=73.2, reward_bound=87.5
12: loss=0.564, reward_mean=82.9, reward_bound=99.0
13: loss=0.567, reward_mean=75.9, reward_bound=94.5
14: loss=0.572, reward_mean=75.2, reward_bound=83.5
15: loss=0.541, reward_mean=72.8, reward_bound=67.0
16: loss=0.561, reward_mean=78.4, reward_bound=89.5
17: loss=0.553, reward_mean=93.4, reward_bound=92.5
18: loss=0.536, reward_mean=83.0, reward_bound=86.0
19: loss=0.542, rewar